In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch % 5 == 0 and epoch != 0:
        return lr * 0.9
    else:
        return lr

data = []
labels = []

for folder in os.listdir("/content/drive/MyDrive/dataset"):
    for file in os.listdir(os.path.join("/content/drive/MyDrive/dataset", folder)):
        img_path = os.path.join("/content/drive/MyDrive/dataset", folder, file)
        try:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))
            img = img.astype("float") / 255.0
            data.append(img)
            labels.append(folder)
        except cv2.error as e:
            print(f"Error processing image {img_path}: {e}")

data = np.array(data)
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
trainY_encoded = label_encoder.fit_transform(trainY)
testY_encoded = label_encoder.transform(testY)

num_classes = len(label_encoder.classes_)

trainY_one_hot = to_categorical(trainY_encoded, num_classes=num_classes)
testY_one_hot = to_categorical(testY_encoded, num_classes=num_classes)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(trainX)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Verify data shapes
print("Shapes - TrainX:", trainX.shape, "TrainY:", trainY_one_hot.shape)
print("Shapes - TestX:", testX.shape, "TestY:", testY_one_hot.shape)

# Verify label encoder classes
print("Label Encoder Classes:", label_encoder.classes_)

# Model summary
model.summary()

# Training
try:
    history = model.fit(datagen.flow(trainX, trainY_one_hot, batch_size=32),
                        validation_data=(testX, testY_one_hot),
                        epochs=20,
                        callbacks=[LearningRateScheduler(scheduler)])
except Exception as e:
    print(f"Error during training: {e}")

# Save the model
model.save("/content/drive/MyDrive/model_multiclass.h5")

# Print history (training and validation loss and accuracy)
if 'history' in locals():
    print(history.history)


Error processing image /content/drive/MyDrive/dataset/acne/.ipynb_checkpoints: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Shapes - TrainX: (72, 224, 224, 3) TrainY: (72, 4)
Shapes - TestX: (18, 224, 224, 3) TestY: (18, 4)
Label Encoder Classes: ['acne' 'acne_scars' 'hyperPigmentation' 'white_patches']
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 111, 111, 32)      128       
 Normalization)                                               

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


{'loss': [8.73965072631836, 7.634267330169678, 7.559305191040039, 9.050756454467773, 9.11449146270752, 8.521060943603516, 5.433544635772705, 7.009028911590576, 6.27396821975708, 4.991941452026367, 5.490191459655762, 7.078676700592041, 4.781212329864502, 5.635561466217041, 6.4200615882873535, 6.334233283996582, 5.926899433135986, 4.34764289855957, 2.358856201171875, 4.235803604125977], 'accuracy': [0.2777777910232544, 0.3888888955116272, 0.3888888955116272, 0.3611111044883728, 0.4861111044883728, 0.4722222089767456, 0.5416666865348816, 0.4166666567325592, 0.4583333432674408, 0.4861111044883728, 0.5694444179534912, 0.4583333432674408, 0.5833333134651184, 0.5972222089767456, 0.5694444179534912, 0.4444444477558136, 0.5138888955116272, 0.5694444179534912, 0.6388888955116272, 0.5694444179534912], 'val_loss': [1.3890196084976196, 1.0439534187316895, 1.4302905797958374, 0.9525148868560791, 1.673545002937317, 2.2923033237457275, 1.5752936601638794, 1.6262186765670776, 1.0835351943969727, 2.3906

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch % 5 == 0 and epoch != 0:
        return lr * 0.9
    else:
        return lr

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Use 'sigmoid' for binary classification

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

classes = ["acne", "acne_scars", "hyperPigmentation", "white_patches"]

for class_name in classes:
    print(f"Training model for class: {class_name}")

    # Data Loading and Preprocessing for the specific class
    data = []
    labels = []

    class_path = os.path.join("/content/drive/MyDrive/dataset", class_name)

    for file in os.listdir(class_path):
        img_path = os.path.join(class_path, file)
        try:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))
            img = img.astype("float") / 255.0
            data.append(img)
            labels.append(1)  # Set label to 1 for the specific class, assuming binary classification
        except cv2.error as e:
            print(f"Error processing image {img_path}: {e}")

    data = np.array(data)
    labels = np.array(labels)

    # Split the data into training and testing sets
    trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.2, random_state=42)

    # Data Augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    datagen.fit(trainX)

    # Build and train the model
    model = build_model()
    model.fit(datagen.flow(trainX, trainY, batch_size=32),
              validation_data=(testX, testY),
              epochs=20,
              callbacks=[LearningRateScheduler(scheduler)])

    # Save the model
    model.save(f"/content/drive/MyDrive/model_{class_name}.h5")

    print(f"Model for class {class_name} trained and saved.")


Training model for class: acne
Error processing image /content/drive/MyDrive/dataset/acne/.ipynb_checkpoints: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Epoch 1/20
2/2 [==============================] - 10s 1s/step - loss: 1.1024 - accuracy: 0.5278 - val_loss: 0.8184 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/20
2/2 [==============================] - 6s 5s/step - loss: 4.7204 - accuracy: 0.7222 - val_loss: 2.0233 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/20
2/2 [==============================] - 6s 5s/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.2519 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/20
2/2 [==============================] - 7s 1s/step - loss: 4.5751e-07 - accuracy: 1.0000 - val_loss: 0.0113 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/20
2/2 [==============================] - 7s 1s/step - loss: 6.9280e-05 - accuracy: 1.0000 - val_loss: 4.6441e-04 - val_accuracy: 1.0000 - 

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch % 5 == 0 and epoch != 0:
        return lr * 0.9
    else:
        return lr

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Use 'sigmoid' for binary classification

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

classes = ["acne", "acne_scars", "hyperPigmentation", "white_patches"]

# Data Loading and Preprocessing for all classes
data = []
labels = []

for class_name in classes:
    print(f"Processing images for class: {class_name}")

    class_path = os.path.join("/content/drive/MyDrive/dataset", class_name)

    for file in os.listdir(class_path):
        img_path = os.path.join(class_path, file)
        try:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))
            img = img.astype("float") / 255.0
            data.append(img)
            labels.append(classes.index(class_name))  # Set label based on class index

        except cv2.error as e:
            print(f"Error processing image {img_path}: {e}")

data = np.array(data)
labels = np.array(labels)

# Split the data into training and testing sets
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.2, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(trainX)

# Build and train the model
model = build_model()
model.fit(datagen.flow(trainX, trainY, batch_size=32),
          validation_data=(testX, testY),
          epochs=20,
          callbacks=[LearningRateScheduler(scheduler)])

# Save the model
model.save("/content/drive/MyDrive/model_all_classes.h5")

print("All models trained and saved.")


Processing images for class: acne
Error processing image /content/drive/MyDrive/dataset/acne/.ipynb_checkpoints: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Processing images for class: acne_scars
Processing images for class: hyperPigmentation
Processing images for class: white_patches
Epoch 1/20
3/3 [==============================] - 17s 6s/step - loss: -0.9257 - accuracy: 0.3056 - val_loss: 0.7170 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 2/20
3/3 [==============================] - 12s 4s/step - loss: -4.7320 - accuracy: 0.2917 - val_loss: 0.6235 - val_accuracy: 0.5556 - lr: 0.0010
Epoch 3/20
3/3 [==============================] - 13s 3s/step - loss: -20.7570 - accuracy: 0.3611 - val_loss: 0.6222 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 4/20
3/3 [==============================] - 13s 4s/step - loss: -28.3932 - accuracy: 0.3472 - val_loss: 0.5729 - val_accuracy: 0.1667 - lr: 0.0010
Epoch 5/20
3/3 [=

In [1]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0


In [3]:
with open('/content/app.py', 'w') as f:
    f.write("""
import streamlit as st
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("/content/drive/MyDrive/model_multiclass.h5")

# Define class labels
class_labels = ["class1", "class2", "class3"]  # Replace with your actual class labels

def preprocess_image(image_path):
    try:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))
        img = img.astype("float") / 255.0
        img = np.expand_dims(img, axis=0)
        return img
    except cv2.error as e:
        st.error(f"Error processing image: {e}")
        return None

def make_prediction(image_path):
    if image_path:
        preprocessed_img = preprocess_image(image_path)
        if preprocessed_img is not None:
            prediction = model.predict(preprocessed_img)
            predicted_class_index = np.argmax(prediction)
            predicted_class = class_labels[predicted_class_index]
            confidence = prediction[0][predicted_class_index]

            st.success(f"Prediction: {predicted_class} (Confidence: {confidence:.2%})")
            st.image(image_path, caption="Uploaded Image", use_column_width=True)
        else:
            st.error("Failed to process the image.")
    else:
        st.warning("Please upload an image.")

st.title("Image Classification App")

uploaded_file = st.file_uploader("Choose an image...", type="jpg")

if st.button("Make Prediction"):
    make_prediction(uploaded_file)
""")


In [6]:
!apt-get update
!apt-get install -y libgl1-mesa-glx


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,582 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,047 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-securi

In [14]:
# Save the Streamlit app code to a file, e.g., app.py
app_code = """
import streamlit as st
import cv2
import numpy as np
import tensorflow as tf

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/all_model.tflite")
interpreter.allocate_tensors()

# Define class labels
class_labels = ["class1", "class2", "class3"]  # Replace with your actual class labels

def preprocess_image(image_path):
    try:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))
        img = img.astype("float") / 255.0
        img = np.expand_dims(img, axis=0)
        return img
    except cv2.error as e:
        st.error(f"Error processing image: {e}")
        return None

def make_prediction(image_path):
    if image_path:
        preprocessed_img = preprocess_image(image_path)
        if preprocessed_img is not None:
            # Perform inference with the TensorFlow Lite model
            input_tensor_index = interpreter.get_input_details()[0]['index']
            interpreter.set_tensor(input_tensor_index, preprocessed_img)
            interpreter.invoke()
            output_tensor_index = interpreter.get_output_details()[0]['index']
            prediction = interpreter.get_tensor(output_tensor_index)

            predicted_class_index = np.argmax(prediction)
            predicted_class = class_labels[predicted_class_index]
            confidence = prediction[0][predicted_class_index]

            st.success(f"Prediction: {predicted_class} (Confidence: {confidence:.2%})")
            st.image(image_path, caption="Uploaded Image", use_column_width=True)
        else:
            st.error("Failed to process the image.")
    else:
        st.warning("Please upload an image.")

st.title("Image Classification App")

uploaded_file = st.file_uploader("Choose an image...", type="jpg")

if st.button("Make Prediction"):
    make_prediction(uploaded_file)
"""

with open('/content/drive/MyDrive/stream_lit.py', 'w') as f:
    f.write(app_code)


In [8]:
import tensorflow as tf

# Load your existing model
model = tf.keras.models.load_model('/content/drive/MyDrive/model_all_classes.h5')

# Convert the model to TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the quantized model
with open('quantized_model.tflite', 'wb') as f:
    f.write(tflite_model)


In [9]:
import tensorflow as tf

# Load your existing model
model = tf.keras.models.load_model('/content/drive/MyDrive/model_all_classes.h5')

# Convert the model to TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the quantized model as all_model.h5
with open('/content/drive/MyDrive/all_model.tflite', 'wb') as f:
    f.write(tflite_model)
